In [1]:
import pandas as pd
import re
from tqdm import tqdm
import math
import numpy as np
import json
from ast import literal_eval

In [2]:
die = pd.read_excel("die_doccano.xlsx")
zangdi = pd.read_excel("zangdi.xlsx")
leng = pd.read_excel("length.xlsx")

In [3]:
def getdoccano(id):
    list = []
    if id in zangdi['content_id'].values:
        table = zangdi[zangdi['content_id'] == id]
        content = table['content'].iloc[0]
        for index in table.index:
            name = table['sent'].loc[index]
            length = len(name)
            start = content.find(name)
            if start != -1:
                end = start + length
                list.append([start, end, "葬地"])
    
    if id in leng['content_id'].values:
        table = leng[leng['content_id'] == id]
        content = table['content'].iloc[0]
        for index in table.index:
            name = table['length'].loc[index]
            length = len(name)
            start = content.find(name)
            if start != -1:
                end = start + length
                list.append([start, end, "享年"])
                
    if id in die['content_id'].values:
        table = die[die['content_id'] == id]
        content = table['content'].iloc[0]
        for index in table.index:
            namelist = literal_eval(table['nianhaofull'].loc[index])
            for name in namelist:
                length = len(name)
                start = content.find(name)
                if start != -1:
                    end = start + length
                    list.append([start, end, "葬时"])
    
    dict = {"id": int(id), "text": content, "label":list}
    dict = json.dumps(dict, ensure_ascii=False)
    return dict

In [5]:
index = sorted(list(set(die["content_id"].values)  |
     set(zangdi["content_id"].values)))
splits = np.array_split(index, 4)

In [6]:
with open('bury1.jsonl', 'w') as f:
    for id in splits[0]:
        f.write(getdoccano(id) + "\n")

with open('bury2.jsonl', 'w') as f:
    for id in splits[1]:
        f.write(getdoccano(id) + "\n")

with open('bury3.jsonl', 'w') as f:
    for id in splits[2]:
        f.write(getdoccano(id) + "\n")
    
with open('bury4.jsonl', 'w') as f:
    for id in splits[3]:
        f.write(getdoccano(id) + "\n")

In [31]:
die[die['content_id'] == 4966982]

,Unnamed: 0,content_id,subject,author,sent,content,nianhao,time,nianhaofull,a
3960,28934,4966982,賈潭,徐鉉,/遘疾還京/ws1ep/保大六年九月二十有一日/ws1ep/卒于江寧永安里官舍/ws1ep/...,公諱潭，字孟澤，洛陽人也。周先同姓，即列國之諸侯；漢得名臣，乃洛陽之才子。攀鱗河北，豈須方面...,永安,1105,['永安'],卒于江寧永安里官舍


In [32]:
die

,Unnamed: 0,content_id,subject,author,sent,content,nianhao,time,nianhaofull,a
0,0,8715497,申之,杜士熊,/間嬰疾/ws1ep/卒於正寢/ws1ep/時嘉定八年四月十二日也/ws1ep/享年四十有一/,〔額文〕宋故都監杜公墓碑〔碑文〕先君姓杜，諱申之，字伯禄，世居嘉興府華亭之章廟。曾祖顯，故不...,嘉定,1216,['嘉定八年四月十二日'],時嘉定八年四月十二日也
1,4,5938065,劉伯盛,劉宗,/淳祐辛亥十一月二十日戊申/ws1ep/前蒼梧司法曲江張君之夫彭城劉氏終於廣之東莞/ws1e...,淳祐辛亥十一月二十日戊申，前蒼梧司法曲江張君之夫彭城劉氏終於廣之東莞，壽六十有六。寶祐丙辰十...,淳祐,1241,['淳祐辛亥十一月二十日'],淳祐辛亥十一月二十日戊申
2,39,5749297,陳龜年,陳亮,/以淳熙十五年四月癸酉卒于家/ws1ep/得年五十有九/,始余出國北門，彌望沮洳之地而帶以一水，岸行不足以容車馬，湖泊往往隨在而有。舟至松江，風濤汹湧...,淳熙,1189,['淳熙十五年四月癸酉'],以淳熙十五年四月癸酉卒于家
3,46,5749291,謝達,陳亮,/以淳熙甲辰五月二十三日卒/ws1ep/得年六十有一/,淳熙三四年間，三山林穎秀實之作邑永康，强敏有幹略，一邑不勞而辦，父老以爲三十年所罕有。劉仲光...,淳熙,1174,['淳熙甲辰五月二十三日'],以淳熙甲辰五月二十三日卒
4,51,5749270,陳良能,陳亮,/性之之配爲胡氏/ws1ep/以淳熙四年六月二十七日卒/ws1ep/明年正月七日/ws1ep...,往嘗論鄉之富人，以陳性之爲第一。吾友徐元德居厚亦知此翁可人意，而樂妻其少女焉。居厚以對策切直...,淳熙,1178,['淳熙四年六月二十七日'],以淳熙四年六月二十七日卒
...,...,...,...,...,...,...,...,...,...,...
3957,28922,4967105,王延紹、王延貞父,徐鉉,/昇元六年夏六月二十有二日/ws1ep/卒于建康翔鸞里之官舍/ws1ep/享年五十有一/,君諱某，字某。其先太原人也。昔者諸侯共職，起末運于髭王；儲后上賓，示靈期于瞽史。緱山維岳，肇...,昇元,943,['昇元六年夏六月二十有二日'],昇元六年夏六月二十有二日
3958,28929,4967079,方訥,徐鉉,/歸全委順/ws1ep/終吉永貞/,公諱訥，字希仁。其先河南人也，後世從官，徙籍新安，支派繁衍，遂爲郡之著姓。迨公數世，皆以儒雅...,永貞,805,['永貞'],終吉永貞
3959,28930,4966978,陶敬宣,徐鉉,/春秋五十有二/ws1ep/保大八年夏四月十有八日卒于位/,公諱敬宣，字文褒。其先尋陽人，因官徙籍，今爲合淝人也。西京作相，開國封侯，於是貽孫南國，主盟...,保大,1129,['保大八年夏四月十有八日'],保大八年夏四月十有八日卒于位
3960,28934,4966982,賈潭,徐鉉,/遘疾還京/ws1ep/保大六年九月二十有一日/ws1ep/卒于江寧永安里官舍/ws1ep/...,公諱潭，字孟澤，洛陽人也。周先同姓，即列國之諸侯；漢得名臣，乃洛陽之才子。攀鱗河北，豈須方面...,永安,1105,['永安'],卒于江寧永安里官舍
